In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#Leo el csv de eventos

events_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/events_up_to_01062018.csv", low_memory=False)

## generacion de features

In [ ]:
features_df = pd.DataFrame()

In [ ]:
features_df['person'] = events_df.person.unique()

In [ ]:
features_df.head()

## Candidad de eventos totales de cada usuario

In [ ]:
df = events_df.groupby('person').event.count()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.rename(columns={'event':'cant_eventos_totales'}, inplace=True)

## Cantidad de cada evento

In [ ]:
df = events_df.groupby('person').event.value_counts()

In [ ]:
df = df.unstack()
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.rename(columns={'ad campaign hit':'cant_ach', 'brand listing':'cant_bl', 'checkout':'cant_ch', 
                            'conversion':'cant_cv', 'generic listing':'cant_gl', 'lead':'cant_l',
                           'search engine hit':'cant_seh', 'searched products':'cant_s_p', 
                            'staticpage':'cant_sp', 'viewed product':'cant_vp', 'visited site':'cant_vs'}, inplace=True)

In [ ]:
features_df.head()

## Cantidad de returnings

In [ ]:
df = events_df.groupby('person').new_vs_returning.value_counts().unstack().drop(columns=['New'])

In [ ]:
features_df = features_df.join(df, on='person')

## Tiempo total 

In [ ]:
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'])

In [ ]:
df = (events_df.groupby('person').timestamp.agg('max') - events_df.groupby('person').timestamp.agg('min'))
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df = features_df.rename(columns={'timestamp':'tiempo_total'})

In [ ]:
features_df[['person', 'tiempo_total']].head()

In [ ]:
features_df.tiempo_total = pd.to_numeric(features_df.tiempo_total, downcast='float')
features_df.tiempo_total.head()

## Tiempo entre new y returning

In [ ]:
df_news = events_df.loc[events_df.new_vs_returning == 'New']
df_news[['person', 'timestamp', 'new_vs_returning']].head()

In [ ]:
df_returnings = events_df.loc[events_df.new_vs_returning == 'Returning']
df_returnings[['person', 'timestamp', 'new_vs_returning']].head()

In [ ]:
df = (df_returnings.groupby('person').timestamp.min() - df_news.groupby('person').timestamp.min())
df = df.fillna(0)
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df = features_df.rename(columns={'timestamp':'tiempo_new_returning'})

In [ ]:
features_df[['person', 'tiempo_new_returning']].head()

In [ ]:
features_df.tiempo_new_returning = pd.to_numeric(features_df.tiempo_new_returning, downcast='float')
features_df.tiempo_new_returning.head()

## Mean entre tiempos de returnings

In [ ]:
df_returnings = events_df.loc[events_df.new_vs_returning == 'Returning']
df_returnings[['person', 'timestamp', 'new_vs_returning']].head()

In [ ]:
df_returnings.groupby('person').head()

## Channel mas frecuente

In [ ]:
def agg_mas_frecuente(x):
    return (x.value_counts().index[0] if(len(x.value_counts())>0) else np.nan)

In [ ]:
df = events_df.groupby('person').channel.agg(lambda x: agg_mas_frecuente(x))
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.columns

In [ ]:
features_df.rename(columns={'channel':'channel_frecuente'}, inplace=True)

In [ ]:
df.value_counts().idxmax()

In [ ]:
features_df.channel_frecuente.fillna(df.value_counts().idxmax(), inplace=True)

In [ ]:
features_df.channel_frecuente.isna().value_counts()

In [ ]:
features_df[['person', 'channel_frecuente']].head()

## Device mas frecuente

In [ ]:
df = events_df.groupby('person').device_type.agg(lambda x: agg_mas_frecuente(x))
df.head()

In [ ]:
features_df = features_df.join(df, on='person')

In [ ]:
features_df.rename(columns={'device_type':'device_frecuente'}, inplace=True)

In [ ]:
df.value_counts().idxmax()

In [ ]:
features_df.channel_frecuente.fillna(df.value_counts().idxmax(), inplace=True)

In [ ]:
features_df.head()

## Support para el storage

In [ ]:
ct = pd.crosstab(events_df.person, events_df.storage)

In [ ]:
ct = ct.astype('bool')

In [ ]:
from mlxtend.frequent_patterns import apriori 

frequent_itemsets = apriori(ct, min_support=0.0, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
frequent_itemsets.head()

In [ ]:
def columna_por_persona(crosstab):
    diccionario = {}
    for person in crosstab.index :
        #print(person)
        lista_value = list()
        for value in crosstab.columns :
            if crosstab.loc[person, value] > 0 :
                lista_value.append(value)
        values = frozenset(lista_value)        
        diccionario[person] = values                
    return diccionario

In [ ]:
result = columna_por_persona(ct)

In [ ]:
#Algoritmo para asignarle el support a la persona segun sus eventos(Toma todos los ev realizados por la persona)
#Result es lo que devuelve storage por persona
def asignar_support(result, frequent_itemsets):
    lista_support = list()
    for value in result.values() :
        #print(value)
        support_value = obtener_support(value, frequent_itemsets)
        #print(support_value)
        lista_support.append(support_value)
    return lista_support    

In [ ]:
def obtener_support(nupla, frequent_itemsets):
    if( len(frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values) == 0 ):
        return 0;
    support_value = frequent_itemsets.loc[ frequent_itemsets['itemsets']==nupla ].support.values[0]
    return support_value  

In [ ]:
lista_support = asignar_support(result, frequent_itemsets)
len(lista_support)

In [ ]:
def lista_personas_f(result):
    lista_personas = list()
    for key in result.keys():
        lista_personas.append(key)
    return lista_personas  

In [ ]:
lista_personas = lista_personas_f(result)
len(lista_personas)

In [ ]:
d = {'person': lista_personas, 'storage_support': lista_support}
df = pd.DataFrame(data=d)

In [ ]:
features_df = features_df.join(df.set_index('person'), on='person', how='left')
features_df.head(3)

## Encoding de las variables categoricas

In [ ]:
import category_encoders as ce

## Channel frecuente

In [ ]:
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

## Device frecuente

In [ ]:
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

## Leo los labels

In [ ]:
labels_df = pd.read_csv("/home/miki_mustard/Desktop/Facultad/Datos/TP2/fiuba-trocafone-tp2-final-set/labels_training_set.csv", low_memory=False)

In [ ]:
#Separo en train y to_predict
train_df = features_df.merge(labels_df, on='person', how='right')
train_df.shape

In [ ]:
to_predict = features_df[~features_df.person.isin(labels_df.person)]
to_predict.shape

##  Random forest

In [ ]:
#Uso https://towardsdatascience.com/random-forest-in-python-24d0893d51c0

In [ ]:
#Separo features de labels

In [ ]:
# Labels: los valores a predecir
#labels = np.array(train_df.label)
labels = train_df.label

#remuevo el label del df
train_df = train_df.drop(columns=['label'])

In [ ]:
train_df.fillna(0, inplace=True)

In [ ]:
# Guardo los nombres de los features para usarlos luego
feature_list = list(train_df.drop(columns=['person']).columns)

In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [ ]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

In [ ]:
#Encoding de las vbles categoricas

#Channel frecuente
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

X_train.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(X_train.channel_frecuente), y_train.label)

X_test.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(X_test.channel_frecuente))

In [ ]:
#Encoding de las vbles categoricas

#Device frecuente
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

X_train.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(X_train.device_frecuente), y_train.label)

X_test.device_frecuente = te_device_frecuente.transform(pd.DataFrame(X_test.device_frecuente))

In [ ]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

In [ ]:
#Aca ver lo del baseline

In [ ]:
#Train model

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.label);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(y_test,predictions)

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

# Pull out one tree from the forest
tree = rf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
graph.write_png('tree.png')

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

In [ ]:
#Obtengo la importancia de los features

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

In [ ]:
#Saco las columnas con importancia menor o igual a 0.05 de las cant y recalculo
new_train_df = train_df.drop(columns=['cant_l', 'cant_sp', 'cant_cv', 'cant_vs', 'cant_s_p', 'cant_seh', 'cant_gl',
                                     'device_frecuente', 'channel_frecuente', 'Returning', 'cant_bl', 'cant_ach'])

In [ ]:
# Guardo los nombres de los features para usarlos luego
feature_list = list(new_train_df.drop(columns=['person']).columns)

In [ ]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

#Separo en test y train
X_train, X_test, y_train, y_test = \
    train_test_split(new_train_df.drop(columns=['person']) , labels, test_size=0.2, random_state=123)

In [ ]:
X_train.reset_index(inplace=True)
X_train.drop(columns='index', inplace=True)
X_test.reset_index(inplace=True)
X_test.drop(columns='index', inplace=True)
y_train = pd.DataFrame(y_train)

y_train.reset_index(inplace=True)
y_train.drop(columns='index', inplace=True)
y_test = pd.DataFrame(y_test)
y_test.reset_index(inplace=True)
y_test.drop(columns='index', inplace=True)

to_predict = pd.DataFrame(to_predict)
to_predict.reset_index(inplace=True)
to_predict.drop(columns='index', inplace=True)

In [ ]:
#Encoding de las vbles categoricas

#Channel frecuente
te_channel_frecuente = ce.TargetEncoder(handle_unknown='ignore')

X_train.channel_frecuente = te_channel_frecuente.fit_transform(pd.DataFrame(X_train.channel_frecuente), y_train.label)

X_test.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(X_test.channel_frecuente))

In [ ]:
#Encoding de las vbles categoricas

#Device frecuente
te_device_frecuente = ce.TargetEncoder(handle_unknown='ignore')

X_train.device_frecuente = te_device_frecuente.fit_transform(pd.DataFrame(X_train.device_frecuente), y_train.label)

X_test.device_frecuente = te_device_frecuente.transform(pd.DataFrame(X_test.device_frecuente))

In [ ]:
#Train model

# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

# Train the model on training data
rf.fit(X_train, y_train.label);

In [ ]:
# Use the forest's predict method on the test data
predictions = rf.predict(X_test)

In [ ]:
roc_auc_score(y_test,predictions)

In [ ]:
#Obtengo la importancia de los features

# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

## Prediccion

In [ ]:
to_predict.drop(columns='person').columns

In [ ]:
to_predict = to_predict.drop(columns=['cant_l', 'cant_sp', 'cant_cv', 'cant_vs', 'cant_s_p', 'cant_seh', 'cant_gl'])

In [ ]:
to_predict.fillna(0, inplace=True)

In [ ]:
to_predict.channel_frecuente = te_channel_frecuente.transform(pd.DataFrame(to_predict.drop(columns='person').channel_frecuente))

In [ ]:
to_predict.device_frecuente = te_device_frecuente.transform(pd.DataFrame(to_predict.drop(columns='person').device_frecuente))

In [ ]:
preds_posta = rf.predict(to_predict.drop(columns=['person']))

In [ ]:
preds_posta

## CSV a publicar

In [ ]:
to_publish = pd.DataFrame()

In [ ]:
to_publish['person'] = to_predict.person
to_publish.shape

In [ ]:
roc_auc_score(y_test,predictions)to_publish['label'] = preds_posta
to_publish.shape

In [ ]:
to_publish.head()

In [ ]:
to_publish.loc[ to_publish.label > 1 ]

In [ ]:
to_publish.to_csv('28_11_1.csv', index=False)